In [2]:
import torch 
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [26]:
transform = transforms.Compose([
    transforms.ToTensor(), # convert image to tensor
    # standardise the dimensions of the image to 100x100
    transforms.Resize((100, 100)),
    # normalise the image by setting the mean and std to 0.5
    transforms.Normalize(mean=0.5, std=0.5)
])


train_dataset = datasets.Flowers102(root='../data', split='test', download=True, transform=transform) 
val_dataset = datasets.Flowers102(root='../data', split='val', download=True, transform=transform) 
test_dataset = datasets.Flowers102(root='../data', split='train', download=True, transform=transform) 
# NOTE: Due to a bug with the Flowers102 dataset, the train and test splits are swapped

In [28]:
len(train_dataset), len(val_dataset), len(test_dataset)

(6149, 1020, 1020)

In [30]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [34]:
# labels are already loaded inside the dataset
for i, (inputs, labels) in enumerate(train_dataloader):
    print("Batch", i)
    print("Labels:", labels)
    break

Batch 0
Labels: tensor([101,  50,  37,  88,  76,  96,  25,  52,  64,  37,  99,  93, 100, 101,
         87,  84,  15,  27,  71,  75,  68,  77,  88,  93,  76,  73,  71,  79,
         46,  50,  45,  95])
